# Run previous code to set up dataframe

In [1]:
import pandas as pd
from pandas.io.html import read_html # rendundant import as we have above?
from bs4 import BeautifulSoup # though we didn't really need it this time!
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikitables = read_html(page,  attrs={"class":"wikitable sortable"})
df=pd.DataFrame(wikitables[0])
df = df.iloc[1:]#ignore header row as we're going to use our own column names in dataframe now...
df.columns=['PostalCode', 'Borough', 'Neighborhood']
df2 = df.query('Borough != "Not assigned"').reset_index(drop=True)
df3=df2.groupby('PostalCode', as_index=False).agg(lambda x: ', '.join(set(x.dropna())))
df3.loc[df3['Neighborhood'] == 'Not assigned', 'Neighborhood' ] = df3['Borough']
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
latlong = pd.read_csv("http://cocl.us/Geospatial_data/") 
latlong.columns = ['PostalCode','Latitude','Longitude']
df3 = pd.merge(left=df3, right=latlong, on="PostalCode", how="right")
print("Environment ready for clustering...")

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Environment ready for clustering...


In [2]:
df3.shape

(103, 5)

In [4]:
# The code was removed by Watson Studio for sharing.

Let's cluster...

Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make.
to generate maps to visualize your neighborhoods and how they cluster together.

Now, let's get the top 100 venues that are in York, Weston, Toronto within a radius of 500 meters.

PostalCode	Borough	Neighborhood	Latitude	Longitude
98	M9N	York	Weston	43.706876	-79.518188

In [8]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    43.706876, 
    -79.518188, 
    radius, 
    LIMIT)

In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c0d762b4c1f671924258581'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4faef2abe4b08a2bb98d3382-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/conveniencestore_',
          'suffix': '.png'},
         'id': '4d954b0ea243a5684a65b473',
         'name': 'Convenience Store',
         'pluralName': 'Convenience Stores',
         'primary': True,
         'shortName': 'Convenience Store'}],
       'id': '4faef2abe4b08a2bb98d3382',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Elm Street and King Street',
        'distance': 328,
        'formattedAddress': ['Elm Street and King Street',
         'Toronto ON',
         'Canada'],
        'labeledLatLngs': [{'label

In [10]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Olympic convenience store,Convenience Store,43.704486,-79.515789


# We can see that there aren't many popular places in this area (according to foursquare)

In [12]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


# Let's check other neighborhoods...

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
toronto_venues = getNearbyVenues(names=df3['Neighborhood'],
                                   latitudes=df3['Latitude'],
                                   longitudes=df3['Longitude']
                                  )

Rouge, Malvern
Port Union, Rouge Hill, Highland Creek
Morningside, West Hill, Guildwood
Woburn
Cedarbrae
Scarborough Village
Ionview, East Birchmount Park, Kennedy Park
Clairlea, Oakridge, Golden Mile
Scarborough Village West, Cliffside, Cliffcrest
Cliffside West, Birch Cliff
Scarborough Town Centre, Dorset Park, Wexford Heights
Wexford, Maryvale
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Steeles East, Agincourt North, Milliken, L'Amoreaux East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Oriole, Henry Farm, Fairview
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Wilson Heights, Downsview North, Bathurst Manor
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
River

In [15]:
print(toronto_venues.shape)
toronto_venues.head()

(2241, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Morningside, West Hill, Guildwood",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Morningside, West Hill, Guildwood",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


Venues per neighborhood?

In [16]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",9,9,9,9,9,9
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Berczy Park,53,53,53,53,53,53
Business reply mail Processing Centre969 Eastern,18,18,18,18,18,18
"CFB Toronto, Downsview East",3,3,3,3,3,3
Caledonia-Fairbanks,6,6,6,6,6,6
Canada Post Gateway Processing Centre,11,11,11,11,11,11


# Unique venue categories?

In [17]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 277 uniques categories.


# Analyze each neighborhood...

In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
toronto_onehot.shape

(2241, 277)

# group rows by neighborhood and take mean frequency

In [25]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Alderwood, Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Business reply mail Processing Centre969 Eastern,0.055556,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,"CFB Toronto, Downsview East",0.000000,0.0,0.000000,0.000000,0.333333,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Caledonia-Fairbanks,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667
8,Canada Post Gateway Processing Centre,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Cedarbrae,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [26]:
toronto_grouped.shape

(100, 277)

# Print neighborhood and top 5 venues

In [27]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                    venue  freq
0  Furniture / Home Store  0.25
1                  Lounge  0.25
2          Breakfast Spot  0.25
3            Skating Rink  0.25
4             Yoga Studio  0.00


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.22
1  Skating Rink  0.11
2   Coffee Shop  0.11
3          Pool  0.11
4           Pub  0.11


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4                Motel  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq
0  Fast Food Restaurant  0.08
1    Italian Restaurant  0.08
2           Coffee Shop  0.08
3     Indian Restaurant  0.04
4            Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1        Cocktail Bar  0.06
2          Restaurant  0.06
3  Seafood Restaurant  0.04
4          Steakhouse  0.04


----Busin

# Let's move above result to pandas df

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# Create new Top 10 dataframe

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Furniture / Home Store,Lounge,Skating Rink,Breakfast Spot,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Pharmacy,Pool,Pub,Sandwich Place,Coffee Shop,German Restaurant,Creperie
2,Bayview Village,Chinese Restaurant,Japanese Restaurant,Café,Bank,Drugstore,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store
3,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,Fast Food Restaurant,Hardware Store,Restaurant,Butcher,Café,Pub,Pizza Place,Pharmacy
4,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Seafood Restaurant,Café,Cheese Shop,Farmers Market,Steakhouse,Irish Pub
5,Business reply mail Processing Centre969 Eastern,Yoga Studio,Garden,Smoke Shop,Brewery,Light Rail Station,Farmers Market,Spa,Fast Food Restaurant,Burrito Place,Restaurant
6,"CFB Toronto, Downsview East",Bus Stop,Park,Airport,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
7,Caledonia-Fairbanks,Park,Women's Store,Fast Food Restaurant,Market,Pharmacy,Airport,Dessert Shop,Event Space,Adult Boutique,Ethiopian Restaurant
8,Canada Post Gateway Processing Centre,Hotel,Coffee Shop,Gym / Fitness Center,American Restaurant,Fried Chicken Joint,Sandwich Place,Burrito Place,Middle Eastern Restaurant,Mediterranean Restaurant,Drugstore
9,Cedarbrae,Hakka Restaurant,Athletics & Sports,Fried Chicken Joint,Thai Restaurant,Bakery,Caribbean Restaurant,Bank,Discount Store,Dog Run,Doner Restaurant


# Let's cluster neighborhoods, k=5

In [32]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [77]:
toronto_merged = neighborhoods_venues_sorted

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='left', lsuffix='_left', rsuffix='_right')
toronto_merged = toronto_merged.join(toronto_venues.set_index('Neighborhood'), on='Neighborhood', how='left', lsuffix='_left', rsuffix='_right')


toronto_merged.head() # check the last columns!

,Neighborhood,1st Most Common Venue_left,2nd Most Common Venue_left,3rd Most Common Venue_left,4th Most Common Venue_left,5th Most Common Venue_left,6th Most Common Venue_left,7th Most Common Venue_left,8th Most Common Venue_left,9th Most Common Venue_left,...,8th Most Common Venue_right,9th Most Common Venue_right,10th Most Common Venue_right,Cluster Labels_right,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt,Furniture / Home Store,Lounge,Skating Rink,Breakfast Spot,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,...,Dog Run,Doner Restaurant,Donut Shop,0,43.794200,-79.262029,Panagio's Breakfast & Lunch,43.792370,-79.260203,Breakfast Spot
0,Agincourt,Furniture / Home Store,Lounge,Skating Rink,Breakfast Spot,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,...,Dog Run,Doner Restaurant,Donut Shop,0,43.794200,-79.262029,Twilight,43.791999,-79.258584,Lounge
0,Agincourt,Furniture / Home Store,Lounge,Skating Rink,Breakfast Spot,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,...,Dog Run,Doner Restaurant,Donut Shop,0,43.794200,-79.262029,Canadian Tire,43.790169,-79.260961,Furniture / Home Store
0,Agincourt,Furniture / Home Store,Lounge,Skating Rink,Breakfast Spot,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,...,Dog Run,Doner Restaurant,Donut Shop,0,43.794200,-79.262029,Commander Arena,43.794867,-79.267989,Skating Rink
1,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Pharmacy,Pool,Pub,Sandwich Place,Coffee Shop,German Restaurant,...,Coffee Shop,German Restaurant,Creperie,0,43.602414,-79.543484,Il Paesano Pizzeria & Restaurant,43.601280,-79.545028,Pizza Place


In [81]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[43.794200, -79.262029], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map
markers_colors = []
for lat, lon, nei , cluster in zip(toronto_merged['Neighborhood Latitude'], toronto_merged['Neighborhood Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels_right']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']


In [82]:
#CLUSTER ONE
toronto_merged.loc[toronto_merged['Cluster Labels_right'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,1st Most Common Venue_left,5th Most Common Venue_left,6th Most Common Venue_left,7th Most Common Venue_left,8th Most Common Venue_left,9th Most Common Venue_left,10th Most Common Venue_left,Cluster Labels_left,1st Most Common Venue_right,2nd Most Common Venue_right,...,8th Most Common Venue_right,9th Most Common Venue_right,10th Most Common Venue_right,Cluster Labels_right,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Furniture / Home Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,0,Furniture / Home Store,Lounge,...,Dog Run,Doner Restaurant,Donut Shop,0,43.794200,-79.262029,Panagio's Breakfast & Lunch,43.792370,-79.260203,Breakfast Spot
0,Furniture / Home Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,0,Furniture / Home Store,Lounge,...,Dog Run,Doner Restaurant,Donut Shop,0,43.794200,-79.262029,Twilight,43.791999,-79.258584,Lounge
0,Furniture / Home Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,0,Furniture / Home Store,Lounge,...,Dog Run,Doner Restaurant,Donut Shop,0,43.794200,-79.262029,Canadian Tire,43.790169,-79.260961,Furniture / Home Store
0,Furniture / Home Store,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,0,Furniture / Home Store,Lounge,...,Dog Run,Doner Restaurant,Donut Shop,0,43.794200,-79.262029,Commander Arena,43.794867,-79.267989,Skating Rink
1,Pizza Place,Pool,Pub,Sandwich Place,Coffee Shop,German Restaurant,Creperie,0,Pizza Place,Gym,...,Coffee Shop,German Restaurant,Creperie,0,43.602414,-79.543484,Il Paesano Pizzeria & Restaurant,43.601280,-79.545028,Pizza Place
1,Pizza Place,Pool,Pub,Sandwich Place,Coffee Shop,German Restaurant,Creperie,0,Pizza Place,Gym,...,Coffee Shop,German Restaurant,Creperie,0,43.602414,-79.543484,Timothy's Pub,43.600165,-79.544699,Pub
1,Pizza Place,Pool,Pub,Sandwich Place,Coffee Shop,German Restaurant,Creperie,0,Pizza Place,Gym,...,Coffee Shop,German Restaurant,Creperie,0,43.602414,-79.543484,Toronto Gymnastics International,43.599832,-79.542924,Gym
1,Pizza Place,Pool,Pub,Sandwich Place,Coffee Shop,German Restaurant,Creperie,0,Pizza Place,Gym,...,Coffee Shop,German Restaurant,Creperie,0,43.602414,-79.543484,Tim Hortons,43.602396,-79.545048,Coffee Shop
1,Pizza Place,Pool,Pub,Sandwich Place,Coffee Shop,German Restaurant,Creperie,0,Pizza Place,Gym,...,Coffee Shop,German Restaurant,Creperie,0,43.602414,-79.543484,Subway,43.599262,-79.544340,Sandwich Place
1,Pizza Place,Pool,Pub,Sandwich Place,Coffee Shop,German Restaurant,Creperie,0,Pizza Place,Gym,...,Coffee Shop,German Restaurant,Creperie,0,43.602414,-79.543484,Pizza Pizza,43.605340,-79.547252,Pizza Place


In [83]:
#CLUSTER TWO
toronto_merged.loc[toronto_merged['Cluster Labels_right'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,1st Most Common Venue_left,5th Most Common Venue_left,6th Most Common Venue_left,7th Most Common Venue_left,8th Most Common Venue_left,9th Most Common Venue_left,10th Most Common Venue_left,Cluster Labels_left,1st Most Common Venue_right,2nd Most Common Venue_right,...,8th Most Common Venue_right,9th Most Common Venue_right,10th Most Common Venue_right,Cluster Labels_right,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
50,Tennis Court,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,1,Tennis Court,Playground,...,Dog Run,Doner Restaurant,Drugstore,1,43.689574,-79.383160,Moore Park Tennis Club,43.693289,-79.382900,Tennis Court
50,Tennis Court,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,1,Tennis Court,Playground,...,Dog Run,Doner Restaurant,Drugstore,1,43.689574,-79.383160,Moorevale Park,43.693610,-79.383465,Playground
70,Playground,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,1,Playground,Women's Store,...,Doner Restaurant,Donut Shop,Dumpling Restaurant,1,43.744734,-79.239476,McCowan Park,43.745089,-79.239336,Playground


In [84]:
#CLUSTER THREE
toronto_merged.loc[toronto_merged['Cluster Labels_right'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,1st Most Common Venue_left,5th Most Common Venue_left,6th Most Common Venue_left,7th Most Common Venue_left,8th Most Common Venue_left,9th Most Common Venue_left,10th Most Common Venue_left,Cluster Labels_left,1st Most Common Venue_right,2nd Most Common Venue_right,...,8th Most Common Venue_right,9th Most Common Venue_right,10th Most Common Venue_right,Cluster Labels_right,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
72,Cafeteria,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Dessert Shop,2,Cafeteria,Women's Store,...,Donut Shop,Dumpling Restaurant,Dessert Shop,2,43.75749,-79.374714,Cafeteria,43.755862,-79.370649,Cafeteria


In [85]:
#CLUSTER FOUR
toronto_merged.loc[toronto_merged['Cluster Labels_right'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,1st Most Common Venue_left,5th Most Common Venue_left,6th Most Common Venue_left,7th Most Common Venue_left,8th Most Common Venue_left,9th Most Common Venue_left,10th Most Common Venue_left,Cluster Labels_left,1st Most Common Venue_right,2nd Most Common Venue_right,...,8th Most Common Venue_right,9th Most Common Venue_right,10th Most Common Venue_right,Cluster Labels_right,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
47,Park,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,3,Park,Women's Store,...,Doner Restaurant,Donut Shop,Dumpling Restaurant,3,43.688905,-79.554724,Wincott Park,43.689301,-79.551802,Park
57,River,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,3,River,Park,...,Dog Run,Doner Restaurant,Drugstore,3,43.653654,-79.506944,Lambton-Kingsway Park,43.657085,-79.508145,Park
57,River,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,3,River,Park,...,Dog Run,Doner Restaurant,Drugstore,3,43.653654,-79.506944,Humber River Falls,43.655617,-79.501702,River
66,Park,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,3,Park,Playground,...,Diner,Discount Store,Dog Run,3,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
66,Park,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,3,Park,Playground,...,Diner,Discount Store,Dog Run,3,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
66,Park,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,3,Park,Playground,...,Diner,Discount Store,Dog Run,3,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
66,Park,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,3,Park,Playground,...,Diner,Discount Store,Dog Run,3,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
76,Park,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,3,Park,Playground,...,Dog Run,Doner Restaurant,Donut Shop,3,43.815252,-79.284577,Port Royal Park,43.815477,-79.289773,Park
76,Park,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,3,Park,Playground,...,Dog Run,Doner Restaurant,Donut Shop,3,43.815252,-79.284577,Milliken Public School Playground,43.815383,-79.289867,Playground


In [86]:
#CLUSTER FIVE
toronto_merged.loc[toronto_merged['Cluster Labels_right'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,1st Most Common Venue_left,5th Most Common Venue_left,6th Most Common Venue_left,7th Most Common Venue_left,8th Most Common Venue_left,9th Most Common Venue_left,10th Most Common Venue_left,Cluster Labels_left,1st Most Common Venue_right,2nd Most Common Venue_right,...,8th Most Common Venue_right,9th Most Common Venue_right,10th Most Common Venue_right,Cluster Labels_right,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
68,Fast Food Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,4,Fast Food Restaurant,Women's Store,...,Doner Restaurant,Donut Shop,Dumpling Restaurant,4,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant


# DONE